# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk

import pandas as pd
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
y = df.loc[:,'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #Normalize text
    norm_words = re.sub(r'[^a-zA-Z0–9]',' ',text)
 
    #Tokenze words
    words = word_tokenize(norm_words)

    #Stop words 
    words = [w for w in words if w not in stopwords.words('english')]

    #Lemmatize
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
 
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

In [6]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
# Get results and add them to a dataframe.
def get_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for cat in y_test.columns:
        #print(classification_report(y_test[cat], y_pred[:,num]))
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', cat)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
results = get_results(y_test, y_pred)
results

Aggregated f_score: 0.931226829604
Aggregated precision: 0.93294580927
Aggregated recall: 0.943561489016


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: Undefin

,Category,f_score,precision,recall
1,related,0.782167,0.780867,0.793388
2,request,0.875325,0.881228,0.888366
3,offer,0.993139,0.990866,0.995423
4,aid_related,0.741577,0.744751,0.746472
5,medical_help,0.896771,0.898799,0.921424
6,medical_products,0.929613,0.939679,0.949523
7,search_and_rescue,0.966044,0.967324,0.975969
8,security,0.972558,0.963715,0.981564
9,military,0.957318,0.957722,0.969231
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
from sklearn.model_selection import GridSearchCV

parameters = {'clf__estimator__max_depth': [5, 10],
              'clf__estimator__min_samples_leaf':[2]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
results2 = get_results(y_test, y_pred)
results2

Aggregated f_score: 0.894967154655
Aggregated precision: 0.886077540469
Aggregated recall: 0.927191495373


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel

,Category,f_score,precision,recall
1,related,0.648901,0.569243,0.754482
2,request,0.753797,0.689966,0.830642
3,offer,0.993139,0.990866,0.995423
4,aid_related,0.469606,0.709729,0.603051
5,medical_help,0.880036,0.844343,0.918881
6,medical_products,0.921005,0.896531,0.946853
7,search_and_rescue,0.963533,0.951772,0.975588
8,security,0.972621,0.963717,0.981691
9,military,0.95352,0.938669,0.968849
10,child_alone,1,1,1


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
from sklearn.tree import DecisionTreeClassifier
import pickle


# testing a pure decision tree classifier
moc = MultiOutputClassifier(DecisionTreeClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)
results = get_results(y_test, y_pred)
results

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


Aggregated f_score: 0.932023976586
Aggregated precision: 0.931086020795
Aggregated recall: 0.933123961618


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


,Category,f_score,precision,recall
1,related,0.767426,0.769479,0.766707
2,request,0.854601,0.854311,0.854898
3,offer,0.988616,0.987053,0.990235
4,aid_related,0.706365,0.705883,0.707049
5,medical_help,0.900892,0.899785,0.902045
6,medical_products,0.938031,0.937419,0.938663
7,search_and_rescue,0.95774,0.956056,0.959567
8,security,0.971,0.969798,0.972231
9,military,0.959768,0.959228,0.96033
10,child_alone,1,1,1


### 9. Export your model as a pickle file

In [12]:
pickle.dump(cv, open('dr_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import re
import nltk
import pickle
import pandas as pd
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

nltk.download(['punkt','stopwords','wordnet'])



def load_data():
    # load data from database
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table('DisasterResponse', engine)
    X = df['message']
    y = df.loc[:,'related':'direct_report']
    return X, y

def tokenize(text):
    #Normalize text
    norm_words = re.sub(r'[^a-zA-Z0–9]',' ',text)
 
    #Tokenze words
    words = word_tokenize(norm_words)

    #Stop words 
    words = [w for w in words if w not in stopwords.words('english')]

    #Lemmatize
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words] 
    return lemmed

def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {'clf__estimator__max_depth': [5, 10],
                  'clf__estimator__min_samples_leaf':[2]
    }
    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    model = build_model()
    model.fit(X_train, y_train)    
    pickle.dump(model, open('model.pkl', 'wb'))
    y_pred = model.predict(X_test)
    #display_results(model, y_test, y_pred)

main()